In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [7]:
out='../output/'
inp='../input/monthly/'
pref='dam'

In [9]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,(2016/01/01) 00-01,40.1,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
...,...,...,...
744,(2023/05/31) 23-24,17.94,118
745,Min,14.94,0
746,Max,300.03,270
747,Average,55.167755376344086021505376344,132.9563172043010752688172043


In [10]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
1,(2016/01/01) 00-01,40.10,477.5
2,(2016/01/01) 01-02,40.08,486.6
3,(2016/01/01) 02-03,40.06,495.6
4,(2016/01/01) 03-04,40.08,496.6
5,(2016/01/01) 04-05,40.07,502.9
...,...,...,...
740,(2023/05/31) 19-20,99.98,106.0
741,(2023/05/31) 20-21,34.99,173.0
742,(2023/05/31) 21-22,34.92,156.0
743,(2023/05/31) 22-23,17.98,129.0


In [11]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
1,(2016/01/01) 00-01,40.10,477.5,2016-01-01,0
2,(2016/01/01) 01-02,40.08,486.6,2016-01-01,1
3,(2016/01/01) 02-03,40.06,495.6,2016-01-01,2
4,(2016/01/01) 03-04,40.08,496.6,2016-01-01,3
5,(2016/01/01) 04-05,40.07,502.9,2016-01-01,4
...,...,...,...,...,...
740,(2023/05/31) 19-20,99.98,106.0,2023-05-31,19
741,(2023/05/31) 20-21,34.99,173.0,2023-05-31,20
742,(2023/05/31) 21-22,34.92,156.0,2023-05-31,21
743,(2023/05/31) 22-23,17.98,129.0,2023-05-31,22


In [12]:
dat=dat.drop(['datehour'],1)
dat

C:\Users\kabom\AppData\Local\Temp\ipykernel_78680\4059115739.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dat=dat.drop(['datehour'],1)


,price,volume,date,hour
1,40.10,477.5,2016-01-01,0
2,40.08,486.6,2016-01-01,1
3,40.06,495.6,2016-01-01,2
4,40.08,496.6,2016-01-01,3
5,40.07,502.9,2016-01-01,4
...,...,...,...,...
740,99.98,106.0,2023-05-31,19
741,34.99,173.0,2023-05-31,20
742,34.92,156.0,2023-05-31,21
743,17.98,129.0,2023-05-31,22


In [13]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4
2016-01-01,40.08,486.6,1,2016,1,1,4
2016-01-01,40.06,495.6,2,2016,1,1,4
2016-01-01,40.08,496.6,3,2016,1,1,4
2016-01-01,40.07,502.9,4,2016,1,1,4
...,...,...,...,...,...,...,...
2023-05-31,99.98,106.0,19,2023,5,31,2
2023-05-31,34.99,173.0,20,2023,5,31,2
2023-05-31,34.92,156.0,21,2023,5,31,2


In [14]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2023 5     27   3833.0  114160.940        29.783705     150.07      15.44   
           28   3952.0   97351.440        24.633462     150.06      15.43   
           29   3083.3  134647.620        43.669970     150.00      19.92   
           30   3046.0  123034.400        40.392121     139.95      18.91   
           31   3154.0  130113.030        41.253339     134.99      17.94   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2016 1     1     45.982500      502.9       53.0  328.075000 2016-01-01  
           2     54.114167      486.0       40.0  304.979167 2016-01-02  
           3     54.959167      486.0       40.0  299.945833 2016-01-03  
           4     80.164583      486.0        0.0  185.708333 2016-01-04  
           5     84.503333      436.0        0.0  156.266667 2016-01-05  
...                    ...        ...        ...         ...        ...  
2023 5     27    34.127083      270.0       84.0  159.708333 2023-05-27  
           28    30.878333      267.0       56.0  164.666667 2023-05-28  
           29    45.673000      198.0      108.3  154.165000 2023-05-29  
           30    44.274000      194.0       70.0  152.300000 2023-05-30  
           31    43.540000      222.0      106.0  157.700000 2023-05-31  

[2708 rows x 10 columns]

In [15]:
hourly['cat']='off'

if (pref=='dam' or pref=='idm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2016-01-01,40.10,477.5,0,2016,1,1,4,19147.750,off
2016-01-01,40.08,486.6,1,2016,1,1,4,19502.928,off
2016-01-01,40.06,495.6,2,2016,1,1,4,19853.736,off
2016-01-01,40.08,496.6,3,2016,1,1,4,19903.728,off
2016-01-01,40.07,502.9,4,2016,1,1,4,20151.203,off
...,...,...,...,...,...,...,...,...,...
2023-05-31,99.98,106.0,19,2023,5,31,2,10597.880,evening
2023-05-31,34.99,173.0,20,2023,5,31,2,6053.270,standard
2023-05-31,34.92,156.0,21,2023,5,31,2,5447.520,standard


In [16]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2023 5     27   3833.0  114160.940        29.783705     150.07      15.44   
           28   3952.0   97351.440        24.633462     150.06      15.43   
           29   3083.3  134647.620        43.669970     150.00      19.92   
           30   3046.0  123034.400        40.392121     139.95      18.91   
           31   3154.0  130113.030        41.253339     134.99      17.94   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2016 1     1     45.982500      502.9       53.0  328.075000 2016-01-01  ...   
           2     54.114167      486.0       40.0  304.979167 2016-01-02  ...   
           3     54.959167      486.0       40.0  299.945833 2016-01-03  ...   
           4     80.164583      486.0        0.0  185.708333 2016-01-04  ...   
           5     84.503333      436.0        0.0  156.266667 2016-01-05  ...   
...                    ...        ...        ...         ...        ...  ...   
2023 5     27    34.127083      270.0       84.0  159.708333 2023-05-27  ...   
           28    30.878333      267.0       56.0  164.666667 2023-05-28  ...   
           29    45.673000      198.0      108.3  154.165000 2023-05-29  ...   
           30    44.274000      194.0       70.0  152.300000 2023-05-30  ...   
           31    43.540000      222.0      106.0  157.700000 2023-05-31  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2016 1     1                   3.0                 2.0      3851.5   
           2                   NaN                 NaN      5969.5   
           3                   NaN                 NaN      7198.7   
           4                   3.0                 2.0      3828.0   
           5                   3.0                 2.0      3230.0   
...                            ...                 ...         ...   
2023 5     27                  NaN                 NaN      2907.0   
           28                  NaN                 NaN      3952.0   
           29                  3.0                 2.0       539.0   
           30                  3.0                 2.0       562.0   
           31                  3.0                 2.0       600.0   

                standard_volume  morning_volume  evening_volume   off_value  \
year month day                                                                
2016 1     1             2920.3           993.0           109.0  152306.418   
           2             1350.0             NaN             NaN  271567.021   
           3                NaN             NaN             NaN  351712.733   
           4              464.0           109.0            56.0  179880.680   
           5              368.4           109.0            43.0  136366.420   
...                         ...             ...             ...         ...   
2023 5     27             926.0             NaN             NaN   65397.160   
           28               NaN             NaN             NaN   97351.440   
           29            1841.0           470.0           233.3   10754.790   
           30            1834.0           499.0           151.0   10652.340   
           31            1810.0           527.0           217.0   11722.150   

                standard_value  mo

In [17]:
daily.to_csv(out + pref + '_daily.csv')

In [18]:
hourly.to_csv(out + pref + '_hourly.csv')

In [19]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2016-01-01,0,40.10,477.5,off
2016-01-01,1,40.08,486.6,off
2016-01-01,2,40.06,495.6,off
2016-01-01,3,40.08,496.6,off
2016-01-01,4,40.07,502.9,off
...,...,...,...,...
2023-05-31,19,99.98,106.0,evening
2023-05-31,20,34.99,173.0,standard
2023-05-31,21,34.92,156.0,standard


In [20]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [21]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

C:\Users\kabom\AppData\Local\Temp\ipykernel_78680\3018208690.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)


,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2016,1,3785.351613,211930.289323,106.684000,45.982500,82.458788,362.651613,17.606452,159.959113,...,42.0,55.986949,50.402420,81.002858,77.850006,137.720998,273.188920,33.409524,56.583459,37.350000
1,2016,2,3871.755172,261082.316897,123.798500,75.023333,91.797506,367.334483,23.231034,161.993793,...,42.0,67.432548,56.605271,106.547628,129.026815,156.801247,274.654573,12.280952,75.799614,42.580952
2,2016,3,2904.254839,187713.287290,108.651500,60.115833,88.953989,285.867742,18.232258,122.263199,...,46.0,64.633890,55.295628,98.717668,116.099471,107.341031,212.547024,15.502899,56.748043,34.786957
3,2016,4,1921.116667,118965.331100,114.641667,42.853500,85.579278,183.970000,7.236667,80.344000,...,42.0,61.925094,57.116312,104.192200,104.410448,139.262082,152.162463,7.793651,19.055263,4.438095
4,2016,5,1597.551613,89924.522194,105.676500,28.659583,79.317876,165.183871,4.661290,68.442769,...,44.0,56.288962,49.684509,106.019186,95.368696,140.331232,124.132670,10.831818,18.262963,4.170455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2023,1,3756.100000,207476.822355,125.746000,30.826667,66.319073,242.119355,68.212903,158.344731,...,44.0,55.237300,34.867891,76.215269,80.365304,161.163838,190.220739,148.737879,134.474815,76.284091
85,2023,2,2048.264286,141454.909714,172.351667,38.674583,83.185327,125.603571,30.750000,85.344345,...,40.0,69.060868,35.433622,88.001937,74.896020,256.013855,79.391975,108.190000,92.935081,52.227500
86,2023,3,2612.280645,195453.435742,122.485833,32.796667,79.641546,187.503226,52.383871,110.589382,...,46.0,74.820994,54.429693,85.531430,84.986535,195.753865,123.868452,94.756522,102.901779,84.941304
87,2023,4,2433.493333,153828.368233,146.990417,29.842500,89.322336,181.910000,23.896667,102.345833,...,40.0,63.212981,44.089554,74.574620,76.469423,258.967673,108.837255,112.321667,100.213333,46.405000


In [22]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power,evening_days,morning_days,days
0,2016,1,3785.351613,211930.289323,106.684000,45.982500,82.458788,362.651613,17.606452,159.959113,...,81.002858,77.850006,137.720998,273.188920,33.409524,56.583459,37.350000,21,21,31
1,2016,2,3871.755172,261082.316897,123.798500,75.023333,91.797506,367.334483,23.231034,161.993793,...,106.547628,129.026815,156.801247,274.654573,12.280952,75.799614,42.580952,21,21,29
2,2016,3,2904.254839,187713.287290,108.651500,60.115833,88.953989,285.867742,18.232258,122.263199,...,98.717668,116.099471,107.341031,212.547024,15.502899,56.748043,34.786957,23,23,31
3,2016,4,1921.116667,118965.331100,114.641667,42.853500,85.579278,183.970000,7.236667,80.344000,...,104.192200,104.410448,139.262082,152.162463,7.793651,19.055263,4.438095,21,21,30
4,2016,5,1597.551613,89924.522194,105.676500,28.659583,79.317876,165.183871,4.661290,68.442769,...,106.019186,95.368696,140.331232,124.132670,10.831818,18.262963,4.170455,22,22,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2023,1,3756.100000,207476.822355,125.746000,30.826667,66.319073,242.119355,68.212903,158.344731,...,76.215269,80.365304,161.163838,190.220739,148.737879,134.474815,76.284091,22,22,31
85,2023,2,2048.264286,141454.909714,172.351667,38.674583,83.185327,125.603571,30.750000,85.344345,...,88.001937,74.896020,256.013855,79.391975,108.190000,92.935081,52.227500,20,20,28
86,2023,3,2612.280645,195453.435742,122.485833,32.796667,79.641546,187.503226,52.383871,110.589382,...,85.531430,84.986535,195.753865,123.868452,94.756522,102.901779,84.941304,23,23,31
87,2023,4,2433.493333,153828.368233,146.990417,29.842500,89.322336,181.910000,23.896667,102.345833,...,74.574620,76.469423,258.967673,108.837255,112.321667,100.213333,46.405000,20,20,30


In [23]:
monthly=monthly.set_index(['year','month'])

In [24]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

#monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
#monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
#monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
#monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [25]:
monthly.to_csv(out + pref + '_monthly.csv')

In [26]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [27]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

C:\Users\kabom\AppData\Local\Temp\ipykernel_78680\2677402245.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)


,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,off_price_mean,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,186358.718770,52.925088,144.549234,33.576008,62.719537,269.341129,50.659677,155.044173,38.017402,...,2752.137500,5.777120e+06,349.375000,272.625000,66.000000,44.000000,203.930966,117.785127,112.626304,62.190833
1,2,177908.735885,61.541059,178.528363,34.991239,72.156050,231.372124,36.427434,128.164222,42.536148,...,2463.412500,5.025922e+06,326.125000,250.250000,60.375000,40.250000,157.212737,106.909667,106.788661,61.319182
2,3,144870.257891,59.799847,154.327056,36.495000,69.256637,187.299194,41.278629,104.414062,42.468411,...,2783.937500,4.490978e+06,342.250000,277.250000,67.500000,45.000000,123.339114,91.240759,92.820488,62.038049
3,4,146992.744000,52.349668,148.733917,29.916167,61.004031,200.567083,47.992500,122.579694,35.748911,...,3411.712500,4.409782e+06,342.500000,263.875000,63.375000,42.250000,134.353105,112.989434,129.722734,80.734548
4,5,133045.753395,49.632105,150.078629,27.094839,58.742299,200.311694,47.297581,113.656186,32.065314,...,2876.037500,4.124418e+06,346.250000,274.500000,66.750000,44.500000,135.851399,93.366762,113.353290,64.309033
5,6,209908.676700,65.979139,165.705714,33.672333,76.634591,261.339524,48.015238,137.636845,37.848891,...,5762.300000,6.297260e+06,332.571429,250.000000,64.714286,64.714286,173.371640,104.736256,125.125411,89.732085
6,7,283849.864977,73.885666,175.861014,37.769355,83.583639,291.731336,62.416129,163.397550,41.656675,...,6798.971429,8.799346e+06,345.714286,255.142857,65.571429,65.571429,203.542110,130.907800,136.779342,102.941952
7,8,305415.149161,70.799860,172.261751,35.621060,82.114194,338.163134,75.628571,189.807177,38.764999,...,7822.400000,9.467870e+06,338.857143,258.571429,67.285714,67.285714,243.930913,150.443697,147.952220,115.658178
8,9,310298.405995,62.988855,148.162381,33.261190,72.400346,368.173333,93.252857,218.037266,37.482779,...,5253.600000,9.308952e+06,339.142857,265.714286,64.285714,42.857143,276.141284,172.092115,169.438902,124.918338
9,10,340968.628641,65.759817,159.001244,35.206728,75.511545,384.445161,93.159447,238.918414,40.618566,...,4256.057143,1.057003e+07,350.285714,272.428571,65.571429,43.714286,298.477516,200.255471,175.223225,97.106824


In [28]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [29]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

C:\Users\kabom\AppData\Local\Temp\ipykernel_78680\1619064361.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)


price      volume         value
month hour                                      
1     0      36.047009  221.465625   7920.015223
      1      34.813750  224.912054   7858.394527
      2      34.317143  226.886161   7838.780433
      3      34.226250  229.041071   7914.075205
      4      34.831169  229.418952   8009.351105
...                ...         ...           ...
12    19    156.063641   72.764977   9048.046899
      20    127.058525   89.339171   7690.097101
      21    102.387972  145.082949  10867.777636
      22     52.743364  220.103226   9422.381429
      23     43.525438  229.725346   9041.481304

[288 rows x 3 columns]

In [30]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [31]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

C:\Users\kabom\AppData\Local\Temp\ipykernel_78680\4270116260.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)


price      volume         value
year month hour                                      
2016 1     0      48.389643  356.957143  17095.535893
           1      48.296071  360.239286  17210.836107
           2      48.238571  362.228571  17278.547857
           3      48.355714  369.696429  17682.875179
           4      48.556129  358.341935  17206.331742
...                     ...         ...           ...
2023 5     19    176.986774   93.119355  15477.834290
           20    119.043871  105.209677  10507.339355
           21     52.096129  127.306452   5436.036613
           22     46.820968  102.880645   3534.006645
           23     29.659355  122.354839   3317.482258

[2136 rows x 3 columns]

In [32]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [33]:
from statsmodels.tsa.stattools import pacf

In [34]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.000000,1.000000,1.000000
1,0.697010,0.804661,0.783831,0.578885
2,0.019783,0.113587,-0.196047,-0.087882
3,0.118088,0.171249,0.149215,0.160861
4,0.025746,0.280628,-0.045415,0.197885
5,-0.093236,0.083524,-0.289020,-0.118127
6,-0.143283,-0.000050,-0.008066,-0.106538
7,0.191646,0.020036,0.337948,0.162024
8,0.185144,0.194386,0.145238,0.109500
9,-0.136196,0.031219,-0.130703,-0.115105


In [35]:
pacfs.to_csv(out + pref + '_pacfs.csv')